# load instance from file (不能用)

In [ ]:
for input, output in zip(inputs, outputs):
            image_id = input["image_id"]
            instances = output["instances"].to(self._cpu_device)            
            boxes = instances.pred_boxes.tensor.detach().numpy()
            scores = instances.scores.tolist()
            #print(scores)
            classes = instances.pred_classes.tolist()
            for box, score, cls in zip(boxes, scores, classes):
                xmin, ymin, xmax, ymax = box
                # The inverse of data loading logic in `datasets/pascal_voc.py`
                xmin += 1
                ymin += 1
                self._predictions[cls].append(
                    f"{image_id} {score:.3f} {xmin:.1f} {ymin:.1f} {xmax:.1f} {ymax:.1f}"
                )

In [100]:
from detectron2.structures.instances import Instances
import json

with open("instance.json", "r") as file:
    data = json.load(file)

# Create an instance of the class using the loaded data
image_size=[720,1280]
instances_byjson = Instances(image_size, **data)

#print(new_obj)

In [101]:
from detectron2.structures.boxes import Boxes
#for bbox in instances.pred_boxes:
bb = Boxes(torch.tensor(instances_byjson.pred_boxes))
instances_byjson.pred_boxes=bb

In [102]:
boxes = instances_byjson.pred_boxes.tensor.detach().numpy()
scores = instances_byjson.scores
#print(scores)
classes = instances_byjson.pred_classes
for box, score, cls in zip(boxes, scores, classes):
    xmin, ymin, xmax, ymax = box
    # The inverse of data loading logic in `datasets/pascal_voc.py`
    xmin += 1
    ymin += 1
    print(box)
    print(score)
    print(cls)
#     self._predictions[cls].append(
#         f"{image_id} {score:.3f} {xmin:.1f} {ymin:.1f} {xmax:.1f} {ymax:.1f}"
#     )

[794.0923 361.8713 904.4684 427.9004]
0.8817
1
[ 38.9078 348.4537 130.1568 400.4138]
0.8343
1
[882.1136 375.4851 967.7128 445.0491]
0.7642
1
[239.4404 347.6568 346.7108 411.8445]
0.7569
1
[ 926.0231  327.0205 1238.7672  483.0041]
0.7501
1
[751.3641 365.1333 815.7677 410.3546]
0.7203
1
[  0.     342.2136  53.5003 401.1723]
0.7182
1
[637.0533 354.0423 669.5773 384.8073]
0.6454
3
[600.0105 358.195  638.0538 386.5943]
0.5772
1
[688.5172 359.7326 734.5107 394.7672]
0.544
1
[214.4859 348.5891 255.0746 377.5786]
0.465
1
[716.677  366.2444 754.6269 400.9683]
0.4545
1
[568.0503 358.7105 589.7067 380.6057]
0.4478
3
[731.0586 366.6981 773.0266 405.6398]
0.4051
1
[580.2347 357.9715 608.6518 379.3144]
0.3762
1
[559.557  355.3174 582.8528 376.6231]
0.3533
3
[231.4804 352.0087 269.7274 380.0815]
0.3531
1
[643.2463 358.5091 678.7308 391.7163]
0.2785
3
[588.9717 358.2669 619.7491 380.9782]
0.2704
1
[557.0608 355.4078 579.1295 373.8378]
0.2468
1
[570.0653 361.8464 594.163  380.3362]
0.2323
1
[574.9289 3

In [103]:
instances_byjson

Instances(num_instances=100, image_height=720, image_width=1280, fields=[pred_boxes: Boxes(tensor([[ 794.0923,  361.8713,  904.4684,  427.9004],
        [  38.9078,  348.4537,  130.1568,  400.4138],
        [ 882.1136,  375.4851,  967.7128,  445.0491],
        [ 239.4404,  347.6568,  346.7108,  411.8445],
        [ 926.0231,  327.0205, 1238.7672,  483.0041],
        [ 751.3641,  365.1333,  815.7677,  410.3546],
        [   0.0000,  342.2136,   53.5003,  401.1723],
        [ 637.0533,  354.0423,  669.5773,  384.8073],
        [ 600.0105,  358.1950,  638.0538,  386.5943],
        [ 688.5172,  359.7326,  734.5107,  394.7672],
        [ 214.4859,  348.5891,  255.0746,  377.5786],
        [ 716.6770,  366.2444,  754.6269,  400.9683],
        [ 568.0503,  358.7105,  589.7067,  380.6057],
        [ 731.0586,  366.6981,  773.0266,  405.6398],
        [ 580.2347,  357.9715,  608.6518,  379.3144],
        [ 559.5570,  355.3174,  582.8528,  376.6231],
        [ 231.4804,  352.0087,  269.7274,  38

## 1. get proposal

In [9]:
from Nb_utils import *
from detectron2.data import build_detection_train_loader



/home/superorange5/.local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [3]:
ratio=1.2
dataset_name = 'VOC2007_bddval1'  
#dataset_name = 'VOC2007_bddvalsmall'
output_csv_file = 'eval_metric_ck2b.csv'
source_list =['kitti','cityscape']

# config
cfg = setup("configs/pt/final_c2b.yaml")

# load test data
test_data_loader = build_detection_test_loader(cfg, dataset_name)

# load annotations
data_annotation = DatasetCatalog.get(dataset_name)

# load model
# model_c2b = load_sourceonlyModel('configs/PFRCNN/PFRCNN_c2b_8cla.yaml', 'output/PFRCNN_c2b_8class/model_final.pth')
# model_k2b = load_TSmodel('k2b', 'final')


data_loader_iter = iter(test_data_loader)
test_data = data_loader_iter.next()



In [4]:
len(data_loader_iter)

5258

* load 8 class city model

In [5]:
model_city_8class = load_sourceonlyModel('configs/PFRCNN/PFRCNN_c2b_8cla.yaml', 'output/PFRCNN_c2b_8class/model_final.pth')


-------- pretrained model loaded ---------


* load 1 class kitti model

In [6]:
model_k2b = load_TSmodel('k2b', 'final')


keep_experiments/k2b/cfg.yaml
keep_experiments/k2b/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [7]:
def get_proposal_roih2(data,model):

    with torch.no_grad():
        (_,  proposals_rpn_unsup_k, proposals_roih_unsup_k, _,) =model(
            data, branch="unsup_data_weak")
    return proposals_roih_unsup_k

### get proposals

In [8]:
proposal_roih_city = get_proposal_roih2(test_data,model_city_8class)
proposal_roih_kitti = get_proposal_roih(test_data,model_k2b)

/home/superorange5/.local/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
proposal_roih_city

[FreeInstances(num_instances=100, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[ 661.9504,  301.5594,  753.9592,  356.5837],
         [  32.4333,  290.3781,  108.4979,  333.6782],
         [ 735.3243,  312.9042,  806.6793,  370.8742],
         [ 199.5960,  289.7140,  289.0160,  343.2038],
         [ 771.9271,  272.5171, 1032.6285,  402.5034],
         [ 626.3325,  304.2777,  680.0189,  341.9622],
         [   0.0000,  285.1780,   44.5975,  334.3102],
         [ 531.0437,  295.0352,  558.1554,  320.6728],
         [ 500.1650,  298.4958,  531.8777,  322.1619],
         [ 573.9436,  299.7772,  612.2836,  328.9727],
         [ 183.7498,  288.7773,  220.6282,  313.2112],
         [ 597.4175,  305.2036,  629.0522,  334.1402],
         [ 473.5232,  298.9254,  491.5758,  317.1714],
         [ 609.4059,  305.5818,  644.3901,  338.0332],
         [ 178.6975,  291.7406,  205.1131,  314.0556],
         [ 483.6801,  298.3095,  507.3683,  316.0953],
         [ 466.4432,  296

In [10]:
len(proposal_roih_city[0])

100

In [43]:
roih_list = []
roih_list.append(proposal_roih_city)
roih_list.append(proposal_roih_kitti)

### separate bbox by classes

In [130]:
def filter_classes(instances):
    boxes = instances.pred_boxes.tensor #.detach().numpy()
    scores =instances.scores.tolist()
    classes = instances.pred_classes.tolist()
    #print(len(classes))
    bbox_by_cls = [[] for _ in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES)]
    #print(bbox_by_cls)
    for box, score, cls in zip(boxes, scores, classes):
        
        # Check if bbox_by_cls[class_index] is empty
        if not bbox_by_cls[cls]:
        # If empty, create a new Boxes object for the class
            bbox_by_cls[cls] = Boxes(box.reshape(1, -1))
        else:
           
        # If not empty, append to the existing Boxes object for the class
            
            bbox_by_cls[cls] = bbox_by_cls[cls].cat([bbox_by_cls[cls],Boxes(box.reshape(1, -1))])
            


    return bbox_by_cls    

In [12]:
instances.image_size

NameError: name 'instances' is not defined

In [128]:
instances = proposal_roih_city[0]
boxes = instances.pred_boxes.tensor #.detach().numpy()
scores =instances.scores.tolist()
pred_classes = instances.pred_classes
    #print(len(classes))

unique_classes = torch.unique(pred_classes)

# Initialize a dictionary to store filter indexes for each class
filter_indexes = {}

# Loop through unique classes
for class_idx in unique_classes:
    # Initialize a boolean tensor with False values
    filter_index = torch.zeros_like(pred_classes, dtype=torch.bool)
    # Set True for elements equal to the current class index
    filter_index[pred_classes == class_idx] = True
    # Store the filter index for the current class
    filter_indexes[class_idx.item()] = filter_index

# Print filter indexes for each class
for class_idx, filter_index in filter_indexes.items():
    print(f"filter_index[{class_idx}] = {filter_index.tolist()}")
    

filter_index[0] = [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]
filter_index[1] = [True, True, True, True, True, True, True, False, True, True, True, True, False, True, True, True, False, False, True, True, True, True, False, True, True, True, True, False, True, False, True, True, False, False, True, False, False, False, False, False, True, True,

In [14]:
filter_indexes

{0: tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False],
        device='cuda:0'),
 1: tensor([ True,  True,  True,  True,  True,  True,  True, False,  True,  True,
          True,  True, False,  True,  True,  True, False, False,  True,  True,
          True,  Tr

In [15]:
roih_sourcei

NameError: name 'roih_sourcei' is not defined

In [16]:
roih_separate_class = []
class_number = 5
roih_sourcei = instances
# Loop through each class
for class_i in range(class_number):
    # Check if filter indexes has class_i
    if class_i in filter_indexes:
        # Get filter index for class_i
        #filter_index = filter_indexes[class_i]
        # Use filter index to select elements from roih_sourcei[class_i]
        #selected_elements = [roih_sourcei[idx] for idx, include in enumerate(filter_index) if include]
        
        #print(class_i)
        #print(selected_elements)
        roih_separate_class.append(roih_sourcei[filter_indexes[class_i]])
    else:
        # Append an empty array if filter indexes does not have class_i
        roih_separate_class.append([])

print(roih_separate_class)

[Instances(num_instances=3, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[ 685.4396,   64.8855,  898.1867,  383.4017],
        [ 735.0998,  160.8517, 1005.6562,  389.7646],
        [ 786.9486,  251.6827, 1042.1509,  402.7314]], device='cuda:0')), scores: tensor([0.0630, 0.0552, 0.0419], device='cuda:0'), pred_classes: tensor([0, 0, 0], device='cuda:0'), scores_logists: tensor([[ 3.2885, -0.3624, -3.6121, -2.7339,  1.8865, -2.7645, -2.8715,  2.3732,
          4.4550],
        [ 2.2547,  0.2572, -2.9336, -1.9545,  1.4638, -2.1497, -2.2676,  1.6367,
          3.4174],
        [ 1.2456,  3.1901, -2.0167, -1.5520,  0.2320, -1.1804, -1.7390,  1.1358,
          0.3912]], device='cuda:0'), boxes_sigma: tensor([[1.2566, 0.2428, 1.3352, 0.5630],
        [1.2964, 0.3385, 1.3228, 0.6389],
        [0.5930, 0.0256, 0.6070, 0.1564]], device='cuda:0')]), Instances(num_instances=51, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[ 661.9504,  301.5594,  75

In [17]:
len(roih_separate_class)

5

In [18]:
instances[filter_indexes[0]]

Instances(num_instances=3, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[ 685.4396,   64.8855,  898.1867,  383.4017],
        [ 735.0998,  160.8517, 1005.6562,  389.7646],
        [ 786.9486,  251.6827, 1042.1509,  402.7314]], device='cuda:0')), scores: tensor([0.0630, 0.0552, 0.0419], device='cuda:0'), pred_classes: tensor([0, 0, 0], device='cuda:0'), scores_logists: tensor([[ 3.2885, -0.3624, -3.6121, -2.7339,  1.8865, -2.7645, -2.8715,  2.3732,
          4.4550],
        [ 2.2547,  0.2572, -2.9336, -1.9545,  1.4638, -2.1497, -2.2676,  1.6367,
          3.4174],
        [ 1.2456,  3.1901, -2.0167, -1.5520,  0.2320, -1.1804, -1.7390,  1.1358,
          0.3912]], device='cuda:0'), boxes_sigma: tensor([[1.2566, 0.2428, 1.3352, 0.5630],
        [1.2964, 0.3385, 1.3228, 0.6389],
        [0.5930, 0.0256, 0.6070, 0.1564]], device='cuda:0')])

In [19]:
A= instances[filter_indexes[1]]
A.image_height

AttributeError: Cannot find field 'image_height' in the given Instances!

In [172]:
A.image_size

(600, 1067)

In [129]:
bbox_by_cls_kitti = filter_classes(proposal_roih_kitti[0])
for i in range(len(bbox_by_cls_kitti)):
    print(len(bbox_by_cls_kitti[i]))


tensor([657.4095, 299.9626, 760.4630, 359.7516], device='cuda:0')
tensor([ 764.5369,  268.9433, 1029.4111,  406.4922], device='cuda:0')
tensor([621.5575, 301.6969, 683.1107, 345.3727], device='cuda:0')
tensor([ 18.3989, 288.0816, 113.0011, 338.0657], device='cuda:0')
tensor([498.4261, 296.0646, 529.8074, 320.0974], device='cuda:0')
tensor([735.2405, 310.4661, 826.4174, 379.4133], device='cuda:0')
tensor([607.1177, 307.0445, 643.3243, 338.7262], device='cuda:0')
tensor([202.2847, 288.7314, 294.1016, 334.5856], device='cuda:0')
tensor([570.1830, 301.5644, 599.9239, 328.2661], device='cuda:0')
tensor([  3.6120, 279.7521,  52.7149, 341.0433], device='cuda:0')
tensor([169.2108, 282.2517, 237.2882, 322.0952], device='cuda:0')
tensor([584.6550, 306.8770, 621.4645, 333.4751], device='cuda:0')
tensor([614.3864, 305.0042, 660.1732, 341.9177], device='cuda:0')
tensor([1002.3969,  347.6329, 1054.4830,  429.4000], device='cuda:0')
tensor([1064.8705,   25.4577, 1065.7255,   97.1590], device='cuda:0'

In [21]:
bbox_by_cls = filter_classes(proposal_roih_city[0])
for i in range(len(bbox_by_cls)):
    print(len(bbox_by_cls[i]))


3
51
0
38
1
0
4
3


In [22]:
len(proposal_roih_city[0])

100

In [23]:
len(proposal_roih_kitti[0])

55

In [24]:
len(bbox_by_cls[1])

51

In [25]:
len(bbox_by_cls_kitti[0])

55

### get match array no gt

In [26]:
def get_match_array_no_gt(proposals_roih_class,batch_size):

    batch_match_array= []
    for data_idx in range(batch_size):
        match_array_source = []
        for i, source_prediction_n in enumerate(proposals_roih_class):
            match_array_source_n =[]
            # others
            #pairwise_sa_sb = []        
            for j in range(source_num):
                if j!=i:
                    sourcen_n_match_other = structures.pairwise_iou(source_prediction_n[data_idx],proposals_roih_class[j][data_idx])
                    #pairwise_sa_sb.append(sourcen_n_match_other)
                    match_array_source_n.append(get_match_array(sourcen_n_match_other))
            match_array_source.append(match_array_source_n)
        batch_match_array.append(match_array_source)
        return  batch_match_array

* roih_list  2*16
    * roih_list[0] =16 free instance


In [34]:
proposals_roih_class = []
proposals_roih_class.append(bbox_by_cls[1])  #number inconsistent
proposals_roih_class.append(bbox_by_cls_kitti[0])

source_num = len(proposals_roih_class)
#batch_size = max(len(proposals_roih_class[0]),len(proposals_roih_class[0]))
#print(batch_size)

mt_src = get_match_array_no_gt(proposals_roih_class,16)

In [42]:
mt_src

[[[[True]], [[True]]]]

In [35]:
def bb_ensemble(mt_src,src_idx):
    source_num = len(mt_src[0])
    df_src = pd.DataFrame()    
    src_array = np.array(mt_src[src_idx]).T
    df_src = pd.DataFrame(src_array)
    df_src['summary'] = df_src.sum(axis=1)
    keep_index = df_src.index[df_src.summary==source_num]
    return keep_index

In [44]:
batch_size=1
source_index=0  # 以誰為基礎  city 0, kitti 1
pesudo_proposals_roih_combined = []

for batch_idx in range(batch_size):
    keep_index = bb_ensemble(mt_src[batch_idx],source_index)
    pesudo_proposals_roih_combined.append(roih_list[source_index][batch_idx][keep_index])

In [45]:
keep_index

Int64Index([0], dtype='int64')

In [46]:
proposals_roih_class[source_index][batch_idx][keep_index]

Boxes(tensor([[661.9504, 301.5594, 753.9592, 356.5837]], device='cuda:0'))

In [47]:
    def process_pseudo_label2(proposals_roih):
        list_instances = []
        for proposal_bbox_inst in proposals_roih:

            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = FreeInstances(image_shape)

            new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor
            pseudo_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            new_proposal_inst.pseudo_boxes = pseudo_boxes
            new_proposal_inst.scores_logists = proposal_bbox_inst.scores_logists
            if proposal_bbox_inst.has('boxes_sigma'):
                new_proposal_inst.boxes_sigma = proposal_bbox_inst.boxes_sigma
            list_instances.append(new_proposal_inst)
        return list_instances

In [181]:
from pt.structures.instances import FreeInstances

output_instance = process_pseudo_label2(pesudo_proposals_roih_combined)

### try merge bbox

In [149]:
instance2 = output_instance[0]
instance1 = output_instance[0]
instance2

FreeInstances(num_instances=1, image_height=600, image_width=1067, fields=[pseudo_boxes: Boxes(tensor([[661.9504, 301.5594, 753.9592, 356.5837]], device='cuda:0')), scores_logists: tensor([[-0.2947,  8.2850, -1.3603, -1.2562, -1.7221, -1.1645, -1.2892, -0.8428,
         -0.6988]], device='cuda:0'), boxes_sigma: tensor([[-1.8157, -2.3274, -1.7632, -2.2488]], device='cuda:0')])

In [67]:
instance1.get_fields().keys()


dict_keys(['pseudo_boxes', 'scores_logists', 'boxes_sigma'])

In [175]:
bbox_each_class = instance1

In [185]:
pred_boxes_all = torch.empty((0, 4)).to("cuda")
scores_logists_all = torch.empty((0)).to("cuda")
boxes_sigma_all = torch.empty((0)).to("cuda")
pred_classes_all = torch.empty((0)).to("cuda")
    
pred_boxes_all = torch.cat((pred_boxes_all, bbox_each_class.pseudo_boxes.tensor),0)
scores_logists_all =torch.cat((scores_logists_all, bbox_each_class.scores_logists),0)

if bbox_each_class.has('boxes_sigma'):
    boxes_sigma_all = torch.cat((boxes_sigma_all, bbox_each_class.boxes_sigma),0)

pred_classes_all = torch.cat((pred_classes_all,torch.tensor([i]*len(new_bbox_loc), device=new_bbox_loc.device)),0)

In [188]:
pred_boxes_all
scores_logists_all
pred_classes_all
boxes_sigma_all

tensor([[-1.8157, -2.3274, -1.7632, -2.2488]], device='cuda:0')

In [133]:
bb1 = instance1.pseudo_boxes
bb1

Boxes(tensor([[661.9504, 301.5594, 753.9592, 356.5837]], device='cuda:0'))

In [154]:
np_array = np.array([[123,456,789,10],[10,20,30,41]])
new_bbox_loc = torch.from_numpy(np_array).to("cuda")

new_boxes1 = Boxes(new_bbox_loc)
new_boxes1

Boxes(tensor([[123., 456., 789.,  10.],
        [ 10.,  20.,  30.,  41.]], device='cuda:0'))

In [162]:
a = torch.cat((a, new_boxes1.tensor),0)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_cat)

In [163]:
a

tensor([])

In [171]:
a = torch.empty((0, 4)).to("cuda")
a = torch.cat((a, bb1.tensor),0)
a

tensor([[661.9504, 301.5594, 753.9592, 356.5837]], device='cuda:0')

In [155]:
new_bbox_loc

tensor([[123, 456, 789,  10],
        [ 10,  20,  30,  41]], device='cuda:0')

In [157]:
new_bbox_loc = torch.cat((new_bbox_loc, bb1.tensor),0)
new_bbox_loc

tensor([[123.0000, 456.0000, 789.0000,  10.0000],
        [ 10.0000,  20.0000,  30.0000,  41.0000],
        [661.9504, 301.5594, 753.9592, 356.5837],
        [661.9504, 301.5594, 753.9592, 356.5837]], device='cuda:0')

In [145]:

for box in new_bbox_loc:
    bbox_by_clsA=[]
    if not bbox_by_clsA:
            # If empty, create a new Boxes object for the class
        bbox_by_clsA = Boxes(box.reshape(1, -1))
    else:

            # If not empty, append to the existing Boxes object for the class

        bbox_by_clsA = bbox_by_clsA.cat([bbox_by_clsA,Boxes(box.reshape(1, -1))])
    print(bbox_by_clsA)

Boxes(tensor([[123., 456., 789.,  10.]], device='cuda:0'))
Boxes(tensor([[10., 20., 30., 41.]], device='cuda:0'))


In [144]:
bbox_by_clsA

Boxes(tensor([[10., 20., 30., 41.]], device='cuda:0'))

In [124]:
new_boxes2 = Boxes()


Boxes(tensor([[661.9504, 301.5594, 753.9592, 356.5837]], device='cuda:0'))

In [126]:
combine = bb1.cat([new_boxes1]) 
combine

Boxes(tensor([[123., 456., 789.,  10.],
        [ 10.,  20.,  30.,  41.]], device='cuda:0'))

In [85]:
instance1.pseudo_boxes.append(instance2.pseudo_boxes)

AttributeError: 'Boxes' object has no attribute 'append'

In [77]:
getattr(instance1, 'pseudo_boxes')+ getattr(instance2, field)


TypeError: unsupported operand type(s) for +: 'Boxes' and 'Boxes'

In [72]:


combined_fields = {}
for field in instance1.get_fields().keys():
    combined_fields[field] = getattr(instance1, field).cat(getattr(instance2, field))

# Create a new Instances object with the combined fields
combined_instance = Instances(instance1.image_size)
for field, value in combined_fields.items():
    combined_instance.set(field, value)

AssertionError: 

## whole flow

In [ ]:
mt_src = self.get_match_array_nogt(roih_list)
            
pesudo_proposals_roih_combined = []
for batch_idx in range(batch_size):
    keep_index = self.bb_ensemble(mt_src[batch_idx],self.source_index)
    pesudo_proposals_roih_combined.append(roih_list[self.source_index][batch_idx][keep_index])

pesudo_proposals_roih_combined_final = self.process_pseudo_label2(pesudo_proposals_roih_combined)

### how to match every class
* 要有一個mapping

In [ ]:
MODEL: 
    ROI_HEADS:
        NUM_CLASSES: 8 最後evaluate的
            
            
必須mapping到最後的
bdd 8 ('truck', 'car', 'rider', 'person', 'train', 'motorcycle', 'bicycle', 'bus')
統一產生一個8的array [0,1,2,3,4,-1,-1,-1]
kitti 5'truck', 'car', 'rider', 'person', 'train)
if kitti 1 car  [-1,0,-1,-1,-1,-1,-1,-1]


每一個class  非-1的count if 2個  拿那兩個來做BB ensemble 


In [ ]:
if cls == 1:
            class_names = ('car',)
        elif cls == 20:
            class_names = (
                "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
                "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
                "pottedplant", "sheep", "sofa", "train", "tvmonitor"
            )
        elif cls == 8:
            class_names = ('truck', 'car', 'rider', 'person', 'train', 'motorcycle', 'bicycle', 'bus')
        elif cls == 7:
            class_names = ('truck', 'car', 'rider', 'person', 'motorcycle', 'bicycle', 'bus')
        elif cls == 5:
            class_names = ('truck', 'car', 'rider', 'person', 'train')

In [1]:
def generate_mapping(source_classes, target_classes):
    mapping = []
    for target_class in target_classes:
        if target_class in source_classes:
            mapping.append(source_classes.index(target_class))
        else:
            mapping.append(-1)
    return mapping


# Define classes for each dataset
cityscape_classes = ('truck', 'car', 'rider', 'person', 'train', 'motorcycle', 'bicycle', 'bus')
kitti_classes = ('truck', 'car', 'rider', 'person', 'train')
sim10k_classes = ('car')
target_classes = ('truck', 'car', 'rider', 'person', 'train', 'motorcycle', 'bicycle', 'bus')

# Generate mappings
cityscape_mapping = generate_mapping(cityscape_classes, target_classes)
kitti_mapping = generate_mapping(kitti_classes, target_classes)
sim10k_mapping = generate_mapping(sim10k_classes, target_classes)

print("Cityscape Mapping:", cityscape_mapping)
print("KITTI Mapping:", kitti_mapping)
print("SIM10K Mapping:", sim10k_mapping)

Cityscape Mapping: [0, 1, 2, 3, 4, 5, 6, 7]
KITTI Mapping: [0, 1, 2, 3, 4, -1, -1, -1]
SIM10K Mapping: [-1, 0, -1, -1, -1, -1, -1, -1]


In [21]:
proposals=[]
proposals.append(bbox_by_cls)
proposals.append(bbox_by_cls_kitti)

In [22]:
proposals

[[Boxes(tensor([[ 685.4396,   64.8855,  898.1867,  383.4017],
          [ 735.0998,  160.8517, 1005.6562,  389.7646],
          [ 786.9486,  251.6827, 1042.1509,  402.7314]], device='cuda:0')),
  Boxes(tensor([[ 661.9504,  301.5594,  753.9592,  356.5837],
          [  32.4333,  290.3781,  108.4979,  333.6782],
          [ 735.3243,  312.9042,  806.6793,  370.8742],
          [ 199.5960,  289.7140,  289.0160,  343.2038],
          [ 771.9271,  272.5171, 1032.6285,  402.5034],
          [ 626.3325,  304.2777,  680.0189,  341.9622],
          [   0.0000,  285.1780,   44.5975,  334.3102],
          [ 500.1650,  298.4958,  531.8777,  322.1619],
          [ 573.9436,  299.7772,  612.2836,  328.9727],
          [ 183.7498,  288.7773,  220.6282,  313.2112],
          [ 597.4175,  305.2036,  629.0522,  334.1402],
          [ 609.4059,  305.5818,  644.3901,  338.0332],
          [ 178.6975,  291.7406,  205.1131,  314.0556],
          [ 483.6801,  298.3095,  507.3683,  316.0953],
          [ 490.

In [64]:
def bb_ensemble(proposals_to_combined):
    print("sources who have this class={}".format(len(proposals_to_combined)))
    for i in range(len(proposals_to_combined)):
        print("number of proposals={}".format(len(proposals_to_combined[i])))
    

* use mapping to get proposals_to _combined (proposal已經拆開class)

In [76]:
source_mapping = [cityscape_mapping, sim10k_mapping]

# Initialize count dictionary for target classes
count = {cls: 0 for cls in target_classes}


proposals_all_classes = []
# Iterate over each class in target classes
for i, target_cls in enumerate(target_classes):
    proposals_roih_class = []
    print("for class {}".format(target_classes[i]))
    # Iterate over each mapping array
    for j, mapping_array in enumerate(source_mapping):
        #print(j)
        #print(mapping_array)
        # Get the index mapped to target class
        mapped_idx = mapping_array[i]
        # If the mapped index is valid (not -1)
        if mapped_idx != -1:
            # Increment count for the corresponding target class
            
            count[target_cls] += 1
            proposals_roih_class.append(proposals[j][mapped_idx])
    if (len(proposals_roih_class)<=1):
        print(len(proposals_roih_class))
    else:
        batch_size = max(len(row) for row in proposals_roih_class)  
        print("batch_size={}".format(batch_size))
        mt_src = get_match_array_no_gt(proposals_roih_class,batch_size)        
        bb_ensemble(proposals_roih_class)
dims = (len(proposals_roih_class), len(proposals_roih_class[0]))
print("Dimensions of proposals_to_combined:", dims)

# Print the counts
for cls, cls_count in count.items():
    print(f'{cls}: {cls_count}')

for class truck
1
for class car
batch_size=55
sources who have this class=2
number of proposals=51
number of proposals=55
for class rider
1
for class person
1
for class train
1
for class motorcycle
1
for class bicycle
1
for class bus
1
Dimensions of proposals_to_combined: (1, 3)
truck: 1
car: 2
rider: 1
person: 1
train: 1
motorcycle: 1
bicycle: 1
bus: 1


In [59]:
for i in range(8):
    print(len(proposals[0][i]))

3
51
0
38
1
0
4
3


In [35]:
for i in range(8):
    print(len(proposals[1][i]))

55
0
0
0
0
0
0
0


### bb ensemble multiple class

* situation 1 : 3 sources, everybody have this class, normaly ensemble ->get_match_array_no_gt
* situation 2: 3 sources, only one of sources have this class ->take it as pseudo label, without avg ->no matching
* situation 3: 3 sources, only two of sources have this class, ensemble 2 ->get_match_array_no_gt(input only two)
* situation 4: 3 source, everybody have this class, but one of them got zero proposal ->drop (use the normal get_match_array_no_gt?)

In [106]:
a=1
b=[0,-1,2]

if a in b:
    print("yes")
    
    

index_of_element = b.index(a)
print("Index of", element_to_find, "in the array:", b)

ValueError: 1 is not in list